In [1]:
import cv2
import cv2
import numpy as np
import glob
import json
import pandas as pd

C:\Users\Admin\AppData\Local\Temp\ipykernel_25892\193887464.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


https://phamdinhkhanh.github.io/2019/11/22/HOG.html

In [2]:
hog = cv2.HOGDescriptor()

SIZE_IMG = (75, 50)

# 1. Khai báo các tham số
cell_size = (8, 8)  # h x w in pixels
block_size = (2, 2)  # h x w in cells
nbins = 9  # number of orientation bins

# 2. Tính toán các tham số truyền vào HOGDescriptor
# winSize: Kích thước của bức ảnh được crop để chia hết cho cell size.
winSize = (SIZE_IMG[0] // cell_size[1] * cell_size[1], SIZE_IMG[1] // cell_size[0] * cell_size[0])
# blockSize: Kích thước của 1 block
blockSize = (block_size[1] * cell_size[1], block_size[0] * cell_size[0])
# blockStride: Số bước di chuyển của block khi thực hiện chuẩn hóa histogram bước 3
blockStride = (cell_size[1], cell_size[0])
print('Kích thước bức ảnh crop theo winSize (pixel): ', winSize)
print('Kích thước của 1 block (pixel): ', blockSize)
print('Kích thước của block stride (pixel): ', blockStride)

# 3. Compute HOG descriptor
hog = cv2.HOGDescriptor(_winSize=winSize,
                        _blockSize=blockSize,
                        _blockStride=blockStride,
                        _cellSize=cell_size,
                        _nbins=nbins)

Kích thước bức ảnh crop theo winSize (pixel):  (72, 48)
Kích thước của 1 block (pixel):  (16, 16)
Kích thước của block stride (pixel):  (8, 8)


In [3]:
# cv2.imread(r"Label_data\LIE\fall001.jpg")
# 4. Load ảnh
img = cv2.imread(r"Label_data\LIE\fall001.jpg")
img = cv2.resize(img, SIZE_IMG)
# 5. Tính toán HOG descriptor
hist = hog.compute(img)
print('Kích thước của HOG descriptor: ', hist.shape)

Kích thước của HOG descriptor:  (1440,)


In [4]:
data_lie = glob.glob("./Label_data/LIE/*")
data_sit = glob.glob("./Label_data/SIT/*")
data_stand = glob.glob("./Label_data/STAND/*")

In [5]:
lie = []
for i in data_lie:
    img = cv2.imread(i)
    img = cv2.resize(img, SIZE_IMG)
    hist = hog.compute(img)
    lie.append(hist)

In [6]:
sit = []
for i in data_sit:
    img = cv2.imread(i)
    img = cv2.resize(img, SIZE_IMG)
    hist = hog.compute(img)
    sit.append(hist)

In [7]:
stand = []
for i in data_stand:
    img = cv2.imread(i)
    img = cv2.resize(img, SIZE_IMG)
    hist = hog.compute(img)
    stand.append(hist)

In [8]:
sit = pd.DataFrame(sit)
lie = pd.DataFrame(lie)
stand = pd.DataFrame(stand)

In [9]:
sit['label'] = [0 for i in range(len(sit))]
stand['label'] = [1 for i in range(len(stand))]
lie['label'] = [2 for i in range(len(lie))]

In [10]:
from sklearn.model_selection import train_test_split


In [142]:
rd = 20
train_lie, test_lie = train_test_split(lie, test_size=0.2, random_state=rd)
train_stand, test_stand = train_test_split(stand, test_size=0.2, random_state=rd)
train_sit, test_sit = train_test_split(sit, test_size=0.4, random_state=rd)

In [143]:
train = pd.concat([train_lie, train_stand, train_sit]).reset_index().sample(frac=1).drop(columns=['index'])
test = pd.concat([test_lie, test_stand, test_sit]).reset_index().sample(frac=1).drop(columns=['index'])

In [144]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,label
659,0.140071,0.165793,0.136037,0.042848,0.130034,0.103751,0.212065,0.121097,0.119413,0.132427,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2241,0.254781,0.038623,0.128778,0.000565,0.178781,0.000512,0.195156,0.059366,0.254781,0.247627,...,0.041264,0.00000,0.000000,0.03736,0.367331,0.367331,0.036600,0.000000,0.041264,0
1808,0.293508,0.025990,0.109516,0.107064,0.293508,0.082186,0.293508,0.109836,0.293508,0.084305,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2351,0.194969,0.233399,0.198148,0.100491,0.075835,0.048486,0.142613,0.056725,0.133017,0.227403,...,0.012350,0.00215,0.007711,0.00036,0.021696,0.000000,0.002322,0.000888,0.012373,0
931,0.170870,0.116679,0.096629,0.098871,0.087679,0.196332,0.224510,0.245090,0.162754,0.134621,...,0.000000,0.00000,0.000000,0.00000,0.034913,0.022721,0.014625,0.003452,0.000000,1


In [145]:
X_train = train.drop('label', axis=1)
y_train = train['label']
X_test = test.drop('label', axis=1)
y_test = test['label']

In [146]:
X_train 

,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
659,0.140071,0.165793,0.136037,0.042848,0.130034,0.103751,0.212065,0.121097,0.119413,0.132427,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2241,0.254781,0.038623,0.128778,0.000565,0.178781,0.000512,0.195156,0.059366,0.254781,0.247627,...,0.082323,0.041264,0.000000,0.000000,0.037360,0.367331,0.367331,0.036600,0.000000,0.041264
1808,0.293508,0.025990,0.109516,0.107064,0.293508,0.082186,0.293508,0.109836,0.293508,0.084305,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2351,0.194969,0.233399,0.198148,0.100491,0.075835,0.048486,0.142613,0.056725,0.133017,0.227403,...,0.006871,0.012350,0.002150,0.007711,0.000360,0.021696,0.000000,0.002322,0.000888,0.012373
931,0.170870,0.116679,0.096629,0.098871,0.087679,0.196332,0.224510,0.245090,0.162754,0.134621,...,0.475665,0.000000,0.000000,0.000000,0.000000,0.034913,0.022721,0.014625,0.003452,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,0.294276,0.170440,0.082407,0.003352,0.294276,0.004784,0.039603,0.171702,0.294276,0.158878,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2219,0.085293,0.025941,0.085563,0.098428,0.143089,0.142897,0.181262,0.214051,0.241266,0.116022,...,0.007252,0.009949,0.013399,0.030433,0.039295,0.310375,0.310375,0.212487,0.145129,0.030569
1909,0.314257,0.010049,0.078166,0.093065,0.314257,0.110612,0.089335,0.014765,0.314257,0.179573,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.958134,0.000000,0.000000,0.000000,0.000000
963,0.304444,0.304444,0.210570,0.026342,0.009775,0.000000,0.000000,0.000000,0.032840,0.304444,...,0.081023,0.144233,0.102080,0.115970,0.076529,0.305685,0.176910,0.184561,0.101352,0.129633


In [147]:
y_test.value_counts(),y_train.value_counts()

(label
 0    646
 1    206
 2    158
 Name: count, dtype: int64,
 label
 0    968
 1    824
 2    632
 Name: count, dtype: int64)

In [148]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [166]:
svm_model = SVC(kernel='poly', max_iter = 60)
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print("F1 score: ", f1_score(y_test, y_pred, average='weighted'))
print(classification_report(y_test, y_pred))

Accuracy:  0.9485148514851485
F1 score:  0.9486944765640389
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       646
           1       0.91      0.93      0.92       206
           2       0.92      0.97      0.94       158

    accuracy                           0.95      1010
   macro avg       0.93      0.95      0.94      1010
weighted avg       0.95      0.95      0.95      1010



c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=60).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [141]:
# Sử dụng mô hình Decision Tree
# Tạo mô hình Decision Tree
dt_model = DecisionTreeClassifier()

# Huấn luyện mô hình trên tập huấn luyện
dt_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = dt_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print("F1 score: ", f1_score(y_test, y_pred, average='weighted'))
# Hiển thị báo cáo phân loại
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8960396039603961
F1 score:  0.8975635955101819
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.89      0.92       646
           1       0.78      0.89      0.83       206
           2       0.86      0.91      0.88       158

    accuracy                           0.90      1010
   macro avg       0.86      0.90      0.88      1010
weighted avg       0.90      0.90      0.90      1010



In [56]:
# Mô hình KNN
knn_model = KNeighborsClassifier(n_neighbors=3)

# Huấn luyện mô hình
knn_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = knn_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print("F1 score: ", f1_score(y_test, y_pred, average='weighted'))
# Hiển thị báo cáo phân loại
print('Classification Report:')
print(classification_report(y_test, y_pred))


Accuracy: 0.9544554455445544
F1 score:  0.9547207891019274
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       323
           1       0.90      0.96      0.93       103
           2       0.94      0.96      0.95        79

    accuracy                           0.95       505
   macro avg       0.94      0.96      0.95       505
weighted avg       0.96      0.95      0.95       505



In [97]:
from sklearn.linear_model import LogisticRegression

# Tạo mô hình Logistic Regression
lr_model = LogisticRegression()

# Huấn luyện mô hình trên tập huấn luyện
lr_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = lr_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.96


In [98]:
# Lưu mô hình
import joblib

joblib.dump(svm_model, 'svm_model_hog.pkl')

['svm_model_hog.pkl']

# Test 

In [99]:
loaded_model = joblib.load('svm_model_hog.pkl')

# Dự đoán
y_pred = loaded_model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))


Accuracy:  0.9663366336633663
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       323
           1       0.93      0.98      0.95       103
           2       0.95      0.96      0.96        79

    accuracy                           0.97       505
   macro avg       0.95      0.97      0.96       505
weighted avg       0.97      0.97      0.97       505

